## Step 1 : Set Environmnent & Load Data

In [ ]:
VERSION = "0.2"

In [2]:
import os
import json

import pandas as pd
import numpy as np

from dotenv import load_dotenv

from scipy.sparse import coo_matrix
from sklearn.decomposition import TruncatedSVD

from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
### Define paths

# Load .env
load_dotenv()

# Input, Output Data 디렉토리 정의
FINE_DATA_DIR = os.getenv("FINE_DATA_DIR")
FEED_DATA_DIR = os.getenv("FEED_DATA_DIR")

if not os.path.exists(FEED_DATA_DIR):
    os.makedirs(FEED_DATA_DIR)

# Users, Products, Interactions 파일 경로 정의
USERS_FILE_PATH = os.path.join(FINE_DATA_DIR, "fine_users.jsonl")
PRODUCTS_FILE_PATH = os.path.join(FINE_DATA_DIR, "fine_products.jsonl")
INTERACTIONS_FILE_PATH = os.path.join(FINE_DATA_DIR, "fine_interactions.jsonl")

In [4]:
### Define Loader Function

def load_jsonl(file_path):
    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        return None

    try:        
        df = pd.read_json(file_path, lines=True)
        print(f"✅ Loaded {len(df):,} rows.")
        return df
    
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

In [5]:
### Load data files

df_users = load_jsonl(USERS_FILE_PATH)
df_products = load_jsonl(PRODUCTS_FILE_PATH)
df_interactions = load_jsonl(INTERACTIONS_FILE_PATH)

✅ Loaded 49,382 rows.
✅ Loaded 642,009 rows.
✅ Loaded 5,831,562 rows.


In [6]:
# Sanity check (Users)
display(df_users)

,uid,country,state,zipcode
0,0,United States,CA,70605
1,1,Mexico,Baja California,22460
2,2,United States,KY,40216
3,3,United States,HI,96740
4,4,United States,CA,71071
...,...,...,...,...
49377,49377,United States,OR,97355
49378,49378,Canada,BC,v0e 1m0
49379,49379,United States,PA,19143
49380,49380,Venezuela,aragua,2103


In [7]:
# Sanity check (Products)
display(df_products)

,pid,name,categories
0,0,TM1856,"[1, 8, 20]"
1,1,QNFSK25S65957,"[1, 14, 1541]"
2,2,VT60201K .,"[1, 8, 22]"
3,3,ICAL-02,"[1, 501, 45]"
4,4,FSD801791-STK,"[1, 13, 44]"
...,...,...,...
642004,642004,MD51274,"[1, 13, 44]"
642005,642005,IT3987-01_B,"[1, 8, 22]"
642006,642006,GY8144,"[3, 69, 172]"
642007,642007,SJE310005_3,"[900, 905]"


In [8]:
# Sanity check (Interactions)
display(df_interactions)

,timestamp,uid,pid
0,2025-10-17 02:37:12,32068,145751
1,2025-10-17 02:37:24,30935,340990
2,2025-10-17 02:37:43,35868,360558
3,2025-10-17 02:39:11,19767,425042
4,2025-10-17 02:39:28,34589,124334
...,...,...,...
5831557,2025-11-14 23:59:58,30652,206834
5831558,2025-11-14 23:59:58,9119,87584
5831559,2025-11-14 23:59:58,11561,393571
5831560,2025-11-14 23:59:58,14661,323023


## Step 2 : Data Cleaning

In [9]:
### Remove rows with missing values (Nulls)

original_len_users = len(df_users)
original_len_products = len(df_products)
original_len_interactions = len(df_interactions)

df_users.dropna(subset=["uid"], inplace=True)
df_products.dropna(subset=["pid"], inplace=True)
df_interactions.dropna(subset=["uid", "pid", "timestamp"], inplace=True)

print(f"✅ Users : {original_len_users:,} -> {len(df_users):,} rows.")
print(f"✅ Products : {original_len_products:,} -> {len(df_products):,} rows.")
print(f"✅ Interactions : {original_len_interactions:,} -> {len(df_interactions):,} rows.")

✅ Users : 49,382 -> 49,382 rows.
✅ Products : 642,009 -> 642,009 rows.
✅ Interactions : 5,831,562 -> 5,831,562 rows.


In [10]:
### Convert Timestamp (UnixTime ms -> Datetime)

df_interactions["timestamp"] = pd.to_datetime(df_interactions["timestamp"], unit="ms")

print("✅ Timestamp converted to datetime objects.")
display(df_interactions.head(7))

✅ Timestamp converted to datetime objects.


,timestamp,uid,pid
0,2025-10-17 02:37:12,32068,145751
1,2025-10-17 02:37:24,30935,340990
2,2025-10-17 02:37:43,35868,360558
3,2025-10-17 02:39:11,19767,425042
4,2025-10-17 02:39:28,34589,124334
5,2025-10-17 02:40:57,5314,108084
6,2025-10-17 02:40:57,5314,108084


## Step 3 : ID Mapping

In [11]:
### Extract unique IDs

unique_users = df_interactions["uid"].unique()
unique_products = df_interactions["pid"].unique()

print(f"Unique Users : {len(unique_users):,} / {len(df_users):,}")
print(f"Unique Products : {len(unique_products):,} / {len(df_products):,}")

Unique Users : 38,820 / 49,382
Unique Products : 539,800 / 642,009


In [12]:
### Create Mapping Dictionaries

# Forward mapping : String ID -> Integer Index (Used for Model Training)
user_to_idx = {user: idx for idx, user in enumerate(unique_users)}
product_to_idx = {product: idx for idx, product in enumerate(unique_products)}

# Reverse Mapping : Integer Index -> String ID (Used for Exporting to Vespa)
idx_to_user = {idx: user for user, idx in user_to_idx.items()}
idx_to_product = {idx: product for product, idx in product_to_idx.items()}

In [13]:
### Apply Mapping to DataFrame

df_interactions["user_idx"] = df_interactions["uid"].map(user_to_idx)
df_interactions["product_idx"] = df_interactions["pid"].map(product_to_idx)

print("✅ ID Mapping Complete.")
display(df_interactions.head(7))

✅ ID Mapping Complete.


,timestamp,uid,pid,user_idx,product_idx
0,2025-10-17 02:37:12,32068,145751,0,0
1,2025-10-17 02:37:24,30935,340990,1,1
2,2025-10-17 02:37:43,35868,360558,2,2
3,2025-10-17 02:39:11,19767,425042,3,3
4,2025-10-17 02:39:28,34589,124334,4,4
5,2025-10-17 02:40:57,5314,108084,5,5
6,2025-10-17 02:40:57,5314,108084,5,5


## Step 4 : Define Tiem Decay Function & Apply Weights

- Weighting Function 변경이 필요한 경우, 이 Step 에서 Function 을 새로 정의하고, weight 를 부여하면 됩니다.

In [14]:
### Define Time Decay Function

def calculate_weights(timestamps_series, reference_date_str="2025-11-14", threshold_days=7, base_weight=1.0, boosted_weight=2.0):
    """
    Timestamp Series 를 입력으로 받아 가중치 시리즈를 반환하는 Weighting Function.

    Parameters :
    - timestamps_series : pd.Series (datetime)
    - reference_date_str : 기준일자의 yyyy-MM-dd 형식 문자열
    - threshold_days : 기준일로부터 가중치를 부여할 기간 (일)
    - base_weight : 기본 가중치
    - boosted_weight : 추가 가중치

    Returns :
    - weights : pd.Series (float)
    """

    ref_date = pd.Timestamp(reference_date_str)
    diff_days = (ref_date - timestamps_series).dt.days

    weights = np.where(diff_days <= threshold_days, boosted_weight, base_weight)

    return weights

In [15]:
### Apply Time Decay Weights

df_interactions["weight"] = calculate_weights(df_interactions["timestamp"])

print("✅ Time Decay Weights Applied.")
print(df_interactions["weight"].value_counts())

display(df_interactions.head(7))

✅ Time Decay Weights Applied.
weight
1.0    3800794
2.0    2030768
Name: count, dtype: int64


,timestamp,uid,pid,user_idx,product_idx,weight
0,2025-10-17 02:37:12,32068,145751,0,0,1.0
1,2025-10-17 02:37:24,30935,340990,1,1,1.0
2,2025-10-17 02:37:43,35868,360558,2,2,1.0
3,2025-10-17 02:39:11,19767,425042,3,3,1.0
4,2025-10-17 02:39:28,34589,124334,4,4,1.0
5,2025-10-17 02:40:57,5314,108084,5,5,1.0
6,2025-10-17 02:40:57,5314,108084,5,5,1.0


## Step 5 : Matrix Creation & Aggregation

- Aggregation 처리에 대한 변경이 필요한 경우, 이 Step 에서 로직을 정의하면 됩니다.

In [16]:
### Aggregate Duplicate Interactions

print(f"Rows before aggregation : {len(df_interactions):,}")
df_matrix = df_interactions.groupby(["user_idx", "product_idx"])["weight"].sum().reset_index()
print(f"Rows after aggregation : {len(df_matrix):,}")

Rows before aggregation : 5,831,562
Rows after aggregation : 2,458,482


In [17]:
### Construct Sparse Matrix (COO Format)

row = df_matrix["user_idx"].values
col = df_matrix["product_idx"].values
data = df_matrix["weight"].values

num_users = len(user_to_idx)
num_products = len(product_to_idx)

user_product_matrix = coo_matrix((data, (row, col)), shape=(num_users, num_products))

In [18]:
### Verify Matrix Properties

print(f"✅ Sparse Matrix Created Successfully.")
print(f"    - Shape : {user_product_matrix.shape} (Users x Products)")
print(f"    - Non-zero elements : {user_product_matrix.nnz:,}")

density = user_product_matrix.nnz / (num_users * num_products) * 100
print(f"    - Matrix Density : {density:.4f}%")

✅ Sparse Matrix Created Successfully.
    - Shape : (38820, 539800) (Users x Products)
    - Non-zero elements : 2,458,482
    - Matrix Density : 0.0117%


## Step 6 : Model Training

In [19]:
### Hyperparameter Setting

# 벡터의 차원인 잠재 요인(Latent Factor) 의 개수 설정
VECTOR_DIMENSION = int(os.getenv("VECTOR_DIMENSION"))
RANDOM_STATE = int(os.getenv("RANDOM_STATE"))

print(f"Training TruncatedSVD Model with k={VECTOR_DIMENSION}, random_state={RANDOM_STATE}")

Training TruncatedSVD Model with k=32, random_state=100


In [20]:
### Initialize Model

# TruncatedSVD : 희소 행렬(Sparse Matrix)을 효율적으로 분해하는 알고리즘
# random_state 값은 실행할 때마다 같은 결과가 나오도록 고정함
svd = TruncatedSVD(n_components=VECTOR_DIMENSION, random_state=RANDOM_STATE)

In [21]:
### Fit & Transform

# fit_transform : 행렬을 분해하고 유저 벡터(U)를 반환
user_factors = svd.fit_transform(user_product_matrix)

# components_ : 학습된 상품 벡터(P) 를 반환 (Transposed 형태이므로 .T 를 해 줌)
# SVD 정의 상 V^T 행렬이 나오는데, sklearn 에서는 componenets_ 에 해당함
product_factors = svd.components_.T

print(f"✅ Model Training Complete.")
print(f"    - User Vectors Shape : {user_factors.shape} (Users x {VECTOR_DIMENSION})")
print(f"    - Product Vectors Shape : {product_factors.shape} (Products x {VECTOR_DIMENSION})")

✅ Model Training Complete.
    - User Vectors Shape : (38820, 32) (Users x 32)
    - Product Vectors Shape : (539800, 32) (Products x 32)


## Step 7 : Model Evaluation

- Model Evaluation 로직의 변경이 필요한 경우, 이 Step 에서 로직을 정의하면 됩니다.

In [22]:
### Explained Variance Ratio

# 모델이 원본 행렬의 정보(분산)를 얼마나 보존하고 있는지 확인
# 값이 너무 낮으면(e.g. 0.1 <) VECTOR_DIM 값을 늘려야 할 수도 있음
explained_variance = svd.explained_variance_ratio_.sum()

print(f"✅ Total Explained Variance Ratio (k={VECTOR_DIMENSION}) : {explained_variance:.4f}")

✅ Total Explained Variance Ratio (k=32) : 0.0818


In [23]:
### Training RMSE (복원 오차)

# 학습된 벡터로 다시 계산했을 때, 원래 점수(weight)와 얼마나 차이가 나는지 확인
# 전체 데이터에서 10만 개의 데이터만 샘플링해서 계산
sample_size = min(100000, len(df_matrix))
df_sample = df_matrix.sample(n=sample_size, random_state=RANDOM_STATE)

# 인덱스 추출
u_idx = df_sample["user_idx"].values
p_idx = df_sample["product_idx"].values
actual_weights = df_sample["weight"].values

# 예측값 계산 (Vector Dot Product)
# Numpy 기능을 이용해 고속으로 내적을 계산 : sum(UserVec & ProductVec)
prediction = np.sum(user_factors[u_idx] * product_factors[p_idx], axis=1)

# RMSE 계산
rmse = sqrt(mean_squared_error(actual_weights, prediction))

print(f"✅ Training RMSE : {rmse:.4f}")
print(f"    - Avg Actual weight : {actual_weights.mean():.2f}")
print(f"    - Avg Predicted Score : {prediction.mean():.2f}")

✅ Training RMSE : 5.1026
    - Avg Actual weight : 3.18
    - Avg Predicted Score : 0.12


## Step 8 : Export to Vespa Feed

In [24]:
### Prepare Metadata Dictionaries (for fast lookup)

meta_users = df_users.set_index("uid").to_dict("index")
meta_products = df_products.set_index("pid").to_dict("index")

In [ ]:
### Define Export Function

def export_vespa_feed(filename, vectors, idx_to_id_map, doc_type, meta_dict):
    filepath = os.path.join(FEED_DATA_DIR, filename)

    with open(filepath, "w", encoding="utf-8") as f:
        count = 0

        for idx, vector in enumerate(vectors):
            # 정수 인덱스 -> 원본 String ID 변환
            obj_id = idx_to_id_map[idx]

            # 기본 필드 구성 (ID + Vector)
            # Vector.tolist() 는 numpy array 를 Python 리스트로 변환함
            fields = {f"{doc_type}_vector": {"values": vector.tolist()}}

            # ID 필드 추가
            if doc_type == "user":
                fields["uid"] = str(obj_id)
            elif doc_type == "product":
                fields["pid"] = str(obj_id)

            # Metadata 병합
            if obj_id in meta_dict:
                meta = meta_dict[obj_id]
                
                if doc_type == "user":
                    if "country" in meta: fields["country"] = meta["country"]
                    if "state" in meta: fields["state"] = meta["state"]
                    if "zipcode" in meta: fields["zipcode"] = meta["zipcode"]
                elif doc_type == "product":
                    if "name" in meta: fields["name"] = meta["name"]
                    if "categories" in meta: fields["categories"] = meta["categories"]

            # Vespa JSON 구조 생성 ("put": "id:namespace:type::id")
            doc = {
                "put": f"id:{doc_type}:{doc_type}::{obj_id}",
                "fields": fields
            }

            f.write(json.dumps(doc, ensure_ascii=False) + "\n")
            count += 1
    
    print(f"✅ Saved {count:,} documents to {filepath}.")

In [26]:
### Excute Export

export_vespa_feed(f"vespa_user_feed.jsonl", user_factors, idx_to_user, "user", meta_users)
export_vespa_feed(f"vespa_product_feed.jsonl", product_factors, idx_to_product, "product", meta_products)

✅ Saved 38,820 documents to /home/vscode/shared/vespa_feed/vespa_user_feed.jsonl.
✅ Saved 539,800 documents to /home/vscode/shared/vespa_feed/vespa_product_feed.jsonl.
